# LMS with Kernel Trick

In [10]:
import numpy as np
import pandas as pd

In [11]:
file = pd.read_csv("weatherHistory.csv")
file

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Cloud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 19:00:00.000 +0200,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 20:00:00.000 +0200,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 21:00:00.000 +0200,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 22:00:00.000 +0200,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


As you can see, there are 12 different input features, but we will remove some, and use only a few others. Also, my pc can't handle this load :P so I'm reducing the dataset size. I'll use only $10k$ data sets. Also, we'll split that into training set and testing set!

In [12]:
#selecting only the required columns
useful_index = [3, 4, 5, 6, 7, 8, 10]
file = file.iloc[:, useful_index]

# getting training set and test set
file_randomized = file.sample(frac=1, random_state=1)

# useable data size
usable_dataset_size = 1200

# training set size
training_set_index = int(0.75 * usable_dataset_size)

# get rid of unnecessary data
file = file_randomized[:usable_dataset_size]

# make training and test set
training_set = file[:training_set_index]
test_set = file[training_set_index:]

## Objective: To predict apparent temperature from other features

### 1. Arrange the data in appropriate format

In [13]:
# Select the columns
input_index = [0, 2, 3, 4, 5, 6]
X = training_set.iloc[:, input_index]
output_index = [1]
Y = training_set.iloc[:, output_index]

# Convert to numpy into suitable format
X = X.to_numpy()
n = X.shape[0]
d = X.shape[1]

Y = Y.to_numpy()

### 3. Construct the training and prediction function

The following function calculates the kernel $\langle \phi\left(x\right), \phi\left(y\right) \rangle$

In [14]:
def kernel_matrix(K, degree=1):
    """K is the matrix of dot product. This applies kernel function to the matrix"""
    result = np.zeros(K.shape)
    for i in range(0, degree+1):
        result += np.power(K, i)
    return result

The following function trains the data

In [15]:
def lms_with_kt(X, Y, alpha, degree = 1, num_iters = 1000):
    """X: nxd vector, Y: nx1 vector, beta: nx1 zero vector, alpha: number, degree: number"""
    # normalize x
    x_min = X.min(axis = 0, keepdims=True)
    x_max = X.max(axis = 0, keepdims=True)
    X = (X - x_min) / (x_max - x_min)
    n = X.shape[0]
    
    # add the column of 1 in the front
    X = np.hstack((np.ones((n, 1)), X))
    
    # make K_matrix (kernel matrix)
    K = np.matmul(X, X.T)
    K = kernel_matrix(K, degree)
    # initialize beta
    beta = np.zeros((n, 1))

    # update beta
    for i in range(num_iters):
        beta += alpha * (Y - np.matmul(K, beta))
    print(beta)
        
    def predict(x):
        """x: 1xd matrix"""
        x_norm = (x - x_min) / (x_max - x_min)
        n_predict = x_norm.shape[0]
        x_norm = np.hstack((np.ones((n_predict, 1)), x_norm))
        K_for_prediction = np.matmul(X, x_norm.T)
        K_for_prediction = kernel_matrix(K_for_prediction, degree)
        return np.dot(beta.T, K_for_prediction)

    return predict

### 4. Train the data and predict the results

#### 4.1 Format the test set

In [16]:
# separate inputs and outputs
test_set_inputs = test_set.iloc[:, input_index]
test_set_outputs = test_set.iloc[:, output_index]

# Convert to numpy
test_set_inputs = test_set_inputs.to_numpy()
test_set_outputs = test_set_outputs.to_numpy()

#### 4.2 Training

In [17]:
predictor = lms_with_kt(X, Y, 0.1, 2, 1000)

<ipython-input-15-d7dcc357189b>:20: RuntimeWarning: overflow encountered in matmul
  beta += alpha * (Y - np.matmul(K, beta))
<ipython-input-15-d7dcc357189b>:20: RuntimeWarning: invalid value encountered in add
  beta += alpha * (Y - np.matmul(K, beta))


[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]

#### 4.3 Prediction

In [18]:
results = predictor(test_set_inputs)